# SysML v2 Test Suite

This can notebook can be cleared using menu `Edit --> Clear All Outputs` and then all cells can be
run using menu `Run --> Run All Cells`.

**WARNING**: This file is generated, all modifications here will be ignored.


## File: ../../sysml/src/examples/Analysis Examples/Vehicle Analysis Demo.sysml

In [1]:
package 'Vehicle Analysis Demo' {
	package VehicleQuantities {
	    import ScalarValues::*;
	    import Quantities::*;
	    import UnitsAndScales::*;
	    import ISQ::*;
	    import USCustomaryUnits::*;
	    
	    attribute def DistancePerVolumeUnit :> DerivedUnit {
	    	private attribute distancePF: QuantityPowerFactor[1] { :>> quantity = isq::L; :>> exponent = 1; }
	        private attribute volumePF: QuantityPowerFactor[1] { :>> quantity = isq::L; :>> exponent = -3; }
	        attribute :>> quantityDimension { :>> quantityPowerFactors = {distancePF, volumePF}; }
	    }

	    attribute def DistancePerVolumeValue :> QuantityValue {
	        :>> num : Real;
	        :>> mRef : DistancePerVolumeUnit;
	    }
	    
	    attribute gallon : VolumeUnit = 231.0 * 'in' ** 3;
	    attribute mpg : DistancePerVolumeUnit = 'mi' / gallon;
	}
	
	package VehicleModel {
	    import VehicleQuantities::*;
	    
	    item def Fuel;
	        
	    port def FuelPort {
	        out item fuel: Fuel;
	    }
	    
	    part def FuelTank {
	        attribute volumeMax : VolumeValue;
	        attribute fuelVolume : VolumeValue;
	        attribute fuelLevel : Real = fuelVolume / volumeMax;
	        
	        port fuelInPort : ~FuelPort;
	        port fuelOutPort : FuelPort;
	    }
	    
	    part def Wheel {
	        attribute diameter : LengthValue;
	    }
	
	    part def Vehicle {
	        attribute mass : MassValue;
	        attribute cargoMass : MassValue;
	        
	        attribute wheelDiameter : LengthValue;
	        attribute driveTrainEfficiency : Real;
	        
	        attribute fuelEconomy_city : DistancePerVolumeValue;
	        attribute fuelEconomy_highway : DistancePerVolumeValue;
	
	        port fuelInPort : ~FuelPort;
	    }
	    
	    part vehicle_c1 : Vehicle {
	        port :>> fuelInPort {
	            in item :>> fuel;
	        }
	
	        part fuelTank : FuelTank {
	            port :>> fuelInPort {
	                in item :>> fuel;
	            }
	        }
	
	        bind fuelInPort::fuel = fuelTank::fuelInPort::fuel;
	
	        part wheel : Wheel[4] {
	            :>> diameter = wheelDiameter;
	        }
	    }
	}
	
	package FuelEconomyRequirementsModel {
	    import VehicleQuantities::*;
	
	    requirement def FuelEconomyRequirement {
	        attribute actualFuelEconomy : DistancePerVolumeValue;
	        attribute requiredFuelEconomy : DistancePerVolumeValue;
	
	        require constraint { actualFuelEconomy >= requiredFuelEconomy }
	    }
	
	    requirement cityFuelEconomyRequirement : FuelEconomyRequirement {
	        :>> requiredFuelEconomy = 25@[mpg];
	    }
	
	    requirement highwayFuelEconomyRequirement : FuelEconomyRequirement {
	        :>> requiredFuelEconomy = 30@[mpg];
	    }
	}
	
	package DynamicsModel {
	    import VehicleQuantities::*;
	    
	    calc def Acceleration(p : PowerValue, m : MassValue, v : SpeedValue) : AccelerationValue =
	        p / (m * v);
	    
	    calc def Velocity(v0 : SpeedValue, a : AccelerationValue, dt : TimeValue) : SpeedValue =
	        v0 + a * dt;
	    
	    calc def Position(x0 : LengthValue, v : SpeedValue, dt : TimeValue) : LengthValue =
	        x0 + v * dt;
	    
	    constraint def StraightLineDynamicsEquations(
	        p : PowerValue,
	        m : MassValue,
	        dt : TimeValue,
	        x_i : LengthValue,
	        v_i : SpeedValue,
	        x_f : LengthValue,
	        v_f : SpeedValue,
	        a : AccelerationValue
	    ) {
	        attribute v_avg : SpeedValue = (v_i + v_f)/2;
	
	        a == Acceleration(p, m, v_avg) &
	        v_f == Velocity(v_i, a, dt) &
	        x_f == Position(x_i, v_avg, dt)
	    }
	    
	    action def StraightLineDynamics (
	        in power : PowerValue,
	        in mass : MassValue,
	        in delta_t : TimeValue,
	        in x_in : LengthValue,
	        in v_in : SpeedValue,
	        out x_out : LengthValue,
	        out v_out : SpeedValue,
	        out a_out : AccelerationValue
	    ) {
	        assert constraint dynamics : StraightLineDynamicsEquations (
	            p = power,
	            m = mass,
	            dt = delta_t,
	            x_i = x_in,
	            v_i = v_in,
	            x_f = x_out,
	            v_f = v_out,
	            a = a_out
	        );
	    }
	}
	
	package FuelEconomyAnalysisModel {
	    import VehicleModel::*;
	    import FuelEconomyRequirementsModel::*;
	    import DynamicsModel::*;
	    import BaseFunctions::size;
	    import Collections::SampledFunction;
	    import Collections::SamplePair;
	
		attribute def ScenarioState {
			position : LengthValue;
			velocity : SpeedValue;
		}
		
		attribute def NominalScenario :> SampledFunction {
			attribute def TimeStateRecord :> SamplePair {
				t : TimeValue :>> domainValue;
				s : ScenarioState :>> rangeValue;
			}
			:>> samples : TimeStateRecord;
			n : Natural = size(samples);
		}
		
		analysis def FuelEconomyAnalysis () calculatedFuelEconomy : DistancePerVolumeValue {
			subject vehicle: Vehicle;
			in attribute scenario : NominalScenario;
			in requirement fuelEconomyRequirement : FuelEconomyRequirement;
			
			objective fuelEconomyAnalysisObjective {
				doc 
				/*
				 * The objective of this analysis is to determine whether the
				 * current vehicle design configuration can satisfy the fuel
				 * economy requirement.
				 */
				
				assume constraint {
					vehicle::wheelDiameter == 33@['in'] &
					vehicle::driveTrainEfficiency == 0.4
				}
				
				require fuelEconomyRequirement {
					:>> actualFuelEconomy = calculatedFuelEconomy;
				}
			}
			
		    
			//import scenario::*;
		
			action dynamicsAnalysis(
				in sc: NominalScenario,
				out power : PowerValue[*],
				out acceleration : AccelerationValue[*]	) {
				/*
				 * Solve for the required engine power as a function of time
				 * to support the scenarios.
				 */
				assert constraint straightLineDynamics {
					{1..sc.n-1}->forAll i (
						StraightLineDynamicsEquations (
							p => power[i],
							m => vehicle::mass,
							dt => sc.samples.t[i+1] - sc.samples.t[i],
							x_i => sc.samples.s.position[i],
							v_i => sc.samples.s.velocity[i],
							x_f => sc.samples.s.position[i+1],
							v_f => sc.samples.s.velocity[i+1],
							a => acceleration[i]                    
						)
					)
				}
			}

		    action fuelConsumptionAnalysis(
		        in power : PowerValue[*] = dynamicsAnalysis::power,
		        in acceleration : AccelerationValue[*] = dynamicsAnalysis::acceleration,
		        out fuelEconomy : DistancePerVolumeValue = calculatedFuelEconomy ) {
		        /*
		         * Solve the engine equations to determine how much fuel is
		         * consumed. The engine RPM is a function of the speed of the
		         * vehicle and the gear state.
		         */
	        }
	    }
	}

	
	part vehicleFuelEconomyAnalysisContext {
	    import FuelEconomyAnalysisModel::*;
	
	    requirement vehicleFuelEconomyRequirementsGroup {
	        import FuelEconomyRequirementsModel::*;
	        
	        subject vehicle : Vehicle;
	
	        requirement vehicleFuelEconomyRequirement_city :> cityFuelEconomyRequirement {
	            doc /* The vehicle shall provide a fuel economy that is greater than or equal to
	             * 25 miles per gallon for the nominal city driving scenarios.
	             */
	
	            :>> actualFuelEconomy = vehicle::fuelEconomy_city;
	
	            assume constraint { vehicle::cargoMass == 1000@[lb] }
	        }
	
	        requirement vehicleFuelEconomyRequirement_highway :> highwayFuelEconomyRequirement {
	            doc /* The vehicle shall provide a fuel economy that is greater than or equal to
	             * 30 miles per gallon for the nominal highway driving scenarios.
	             */
	
	            :>> actualFuelEconomy = vehicle::fuelEconomy_highway;
	
	            assume constraint { vehicle::cargoMass == 1000@[lb] }
	        }
	
	    }
	    
	    attribute cityScenario : NominalScenario;
	    attribute highwayScenario : NominalScenario;
	
	    analysis cityFuelEconomyAnalysis : FuelEconomyAnalysis () {
	        subject vehicle = vehicle_c1;
	        attribute :>> scenario = cityScenario;
	        requirement :>> fuelEconomyRequirement = cityFuelEconomyRequirement;
	    }
	
	    analysis highwayFuelEconomyAnalysis : FuelEconomyAnalysis () {
	        subject vehicle = vehicle_c1;
	        attribute :>> scenario = highwayScenario;
	        requirement :>> fuelEconomyRequirement = highwayFuelEconomyRequirement;
	    }
	
	    part vehicle_c1_analysized :> vehicle_c1 {
	        attribute :>> fuelEconomy_city = cityFuelEconomyAnalysis::calculatedFuelEconomy;
	        attribute :>> fuelEconomy_highway = highwayFuelEconomyAnalysis::calculatedFuelEconomy;
	    }
	
	    satisfy vehicleFuelEconomyRequirementsGroup by vehicle_c1_analysized;
	}
}

Package Vehicle Analysis Demo (6fff8ac8-69ee-40ac-9640-a3b633a5e15d)


In [2]:
%viz --style="COMPTREE" --view="Default" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [3]:
%viz --style="COMPTREE" --view="Tree" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [4]:
%viz --style="COMPTREE" --view="State" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [5]:
%viz --style="COMPTREE" --view="Interconnection" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [6]:
%viz --style="COMPTREE" --view="Action" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [7]:
%viz --style="COMPTREE" --view="Sequence" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [8]:
%viz --style="COMPTREE" --view="MIXED" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [9]:
%viz --style="POLYLINE" --view="Default" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [10]:
%viz --style="POLYLINE" --view="Tree" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [11]:
%viz --style="POLYLINE" --view="State" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [12]:
%viz --style="POLYLINE" --view="Interconnection" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [13]:
%viz --style="POLYLINE" --view="Action" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [14]:
%viz --style="POLYLINE" --view="Sequence" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [15]:
%viz --style="POLYLINE" --view="MIXED" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [16]:
%viz --style="LR" --view="Default" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [17]:
%viz --style="LR" --view="Tree" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [18]:
%viz --style="LR" --view="State" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [19]:
%viz --style="LR" --view="Interconnection" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [20]:
%viz --style="LR" --view="Action" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [21]:
%viz --style="LR" --view="Sequence" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [22]:
%viz --style="LR" --view="MIXED" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [23]:
%viz --style="STDCOLOR" --view="Default" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [24]:
%viz --style="STDCOLOR" --view="Tree" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [25]:
%viz --style="STDCOLOR" --view="State" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [26]:
%viz --style="STDCOLOR" --view="Interconnection" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [27]:
%viz --style="STDCOLOR" --view="Action" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [28]:
%viz --style="STDCOLOR" --view="Sequence" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [29]:
%viz --style="STDCOLOR" --view="MIXED" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [30]:
%viz --style="PLANTUML" --view="Default" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [31]:
%viz --style="PLANTUML" --view="Tree" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [32]:
%viz --style="PLANTUML" --view="State" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [33]:
%viz --style="PLANTUML" --view="Interconnection" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [34]:
%viz --style="PLANTUML" --view="Action" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [35]:
%viz --style="PLANTUML" --view="Sequence" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [36]:
%viz --style="PLANTUML" --view="MIXED" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [37]:
%viz --style="DEFAULT" --view="Default" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [38]:
%viz --style="DEFAULT" --view="Tree" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [39]:
%viz --style="DEFAULT" --view="State" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [40]:
%viz --style="DEFAULT" --view="Interconnection" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [41]:
%viz --style="DEFAULT" --view="Action" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [42]:
%viz --style="DEFAULT" --view="Sequence" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [43]:
%viz --style="DEFAULT" --view="MIXED" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [44]:
%viz --style="TB" --view="Default" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [45]:
%viz --style="TB" --view="Tree" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [46]:
%viz --style="TB" --view="State" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [47]:
%viz --style="TB" --view="Interconnection" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [48]:
%viz --style="TB" --view="Action" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [49]:
%viz --style="TB" --view="Sequence" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [50]:
%viz --style="TB" --view="MIXED" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [51]:
%viz --style="ORTHOLINE" --view="Default" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [52]:
%viz --style="ORTHOLINE" --view="Tree" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [53]:
%viz --style="ORTHOLINE" --view="State" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [54]:
%viz --style="ORTHOLINE" --view="Interconnection" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [55]:
%viz --style="ORTHOLINE" --view="Action" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [56]:
%viz --style="ORTHOLINE" --view="Sequence" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'


In [57]:
%viz --style="ORTHOLINE" --view="MIXED" "'Vehicle"

ERROR:Couldn't resolve reference to Element ''Vehicle'
